## Parameters

In [62]:
dataverse = "LocationDb"
dataset_name = "Locations_RTree_Concurrent"
query_count = 10_000
dbHost = "http://localhost:19002"

osm_dataset = "cemetery"
csv_path_local = f"./data/csv/{osm_dataset}.csv"
adm_path_local = f"./data/adm/{osm_dataset}.adm"
adm_path_container = f"/opt/asterixdb/datasets/adm/{osm_dataset}.adm"

max_mbr_width = 0.01 # TODO Update with standard dev
max_mbr_height = 0.01 # TODO Update with standard dev

## Create Database

In [58]:
import requests

create_db_path = "CREATE_DATABASE.sql"
with open(create_db_path, "r") as sql_file:
    create_database_script = sql_file.read()
response = requests.post(f"{dbHost}/query/service", data={'statement': create_database_script})
print(response.text)

{
	"requestID": "72468034-5198-4ae8-9049-d7907ec58d26",
	"plans":{},
	"status": "success",
	"metrics": {
		"elapsedTime": "639.513167ms",
		"executionTime": "609.170917ms",
		"compileTime": "0ns",
		"queueWaitTime": "0ns",
		"resultCount": 0,
		"resultSize": 0,
		"processedObjects": 0,
		"bufferCacheHitRatio": "0.00%",
		"bufferCachePageReadCount": 0
	}
}



## Load Data

In [59]:
load_command = f"""
LOAD DATASET {dataset_name} USING localfs
    (("path"="127.0.0.1:///{adm_path_container}"),("format"="adm"));
"""
response = requests.post(f"{dbHost}/query/service", data={'statement': load_command, 'dataverse': dataverse})
print(response.text)

{
	"requestID": "4faaa694-aef5-4fb2-8c20-5ad89cf1609b",
	"plans":{},
	"status": "success",
	"metrics": {
		"elapsedTime": "6.994365295s",
		"executionTime": "6.993557795s",
		"compileTime": "0ns",
		"queueWaitTime": "0ns",
		"resultCount": 0,
		"resultSize": 0,
		"processedObjects": 0,
		"bufferCacheHitRatio": "0.00%",
		"bufferCachePageReadCount": 0
	}
}



## Plan Query

In [60]:
bounding_box_centers = []
with open(csv_path_local, "r") as csv_file:
    csv_file.readline()  # Skip header
    for line in csv_file:
        if line.startswith("lat"):
            continue
        id, x, y, description = line.strip().split(",")
        bounding_box_centers.append((float(x), float(y)))

bounding_box_centers[:5]

[(71.60207635350525, 54.78643014239917),
 (68.07547055464238, 40.23959944486414),
 (22.71972581720911, 48.451325419849496),
 (29.491294654319063, 46.07657549119183),
 (79.38921734504402, 52.74566659600047)]

In [ ]:
import random
from utils import point, rectangle, create_offset_rectangle
query_benchmark_plan_path = f"commands/benchmark_query_only_plan_{dataverse}_{dataset_name}_{query_count}.sql"

with open(query_benchmark_plan_path, 'w') as f:
    for x, y in bounding_box_centers:
        mbr_width, mbr_height = random.uniform(1, max_mbr_width), random.uniform(1, max_mbr_height)
        start_x, start_y = x - mbr_width/2, y - mbr_height/2
        end_x, end_y = x + mbr_width/2, y + mbr_height/2
        query = f"SELECT * FROM {dataset_name} WHERE spatial_intersect(location, create_rectangle(create_point({start_x},{start_y}), create_point({end_x},{end_y})));"
        f.write(query + "\n") 

with open(query_benchmark_plan_path, 'r') as f:
    for i in range(5):
        print(f.readline().strip())

SELECT * FROM Locations_RTree_Concurrent WHERE spatial_intersect(location, create_rectangle(create_point(71.31078406781276,54.41980426141749), create_point(71.89336863919775,55.15305602338085)));
SELECT * FROM Locations_RTree_Concurrent WHERE spatial_intersect(location, create_rectangle(create_point(67.8349409895974,39.81422499001828), create_point(68.31600011968736,40.66497389971)));
SELECT * FROM Locations_RTree_Concurrent WHERE spatial_intersect(location, create_rectangle(create_point(22.709540859713773,48.30966858035255), create_point(22.729910774704447,48.59298225934644)));
SELECT * FROM Locations_RTree_Concurrent WHERE spatial_intersect(location, create_rectangle(create_point(29.094679816540577,45.736885639879084), create_point(29.88790949209755,46.41626534250458)));
SELECT * FROM Locations_RTree_Concurrent WHERE spatial_intersect(location, create_rectangle(create_point(79.13890947557928,52.324515708393534), create_point(79.63952521450875,53.1668174836074)));


## Execute Queries

In [64]:
from utils import Logger

logs_filename_parts = [dataset_name, query_count]
logger = Logger(logs_filename_parts, True)
with open(query_benchmark_plan_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        response = requests.post(f"{dbHost}/query/service", data={'statement': line, 'dataverse': dataverse,})
        json_response = response.json()
        if response.status_code != 200:
            logger.error({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "metrics": json_response.get("metrics"),
                "error": json_response.get("errors"),
            })
        else:
            logger.log({
                "status": json_response.get("status"),
                "http.status": response.status_code,
                "result_count": len(json_response.get("results", [])) if json_response.get("results") else None,
                "metrics": json_response.get("metrics"),
            })

1745848597 - INFO - {"status": "success", "http.status": 200, "result_count": 55, "metrics": {"elapsedTime": "59.658792ms", "executionTime": "57.4355ms", "compileTime": "19.695208ms", "queueWaitTime": "0ns", "resultCount": 55, "resultSize": 20397, "processedObjects": 110, "bufferCacheHitRatio": "100.00%", "bufferCachePageReadCount": 167}}
1745848597 - INFO - {"status": "success", "http.status": 200, "result_count": 32, "metrics": {"elapsedTime": "17.46975ms", "executionTime": "17.16525ms", "compileTime": "5.375875ms", "queueWaitTime": "0ns", "resultCount": 32, "resultSize": 11891, "processedObjects": 64, "bufferCacheHitRatio": "100.00%", "bufferCachePageReadCount": 98}}
1745848597 - INFO - {"status": "success", "http.status": 200, "result_count": 2, "metrics": {"elapsedTime": "14.783625ms", "executionTime": "14.427209ms", "compileTime": "5.84325ms", "queueWaitTime": "0ns", "resultCount": 2, "resultSize": 740, "processedObjects": 4, "bufferCacheHitRatio": "100.00%", "bufferCachePageRead

KeyboardInterrupt: 